# Algorithm

```
ID3 (Examples, Target_attribute, Attribute)

< Examples are the training examples. Target attribute is the attribute whose value is to be
predicted by the tree. Attributes is a list of other attributes that may be tested by the learned
decision tree. Returns a decision tree that correctly classifies the given Examples.>

*  Create a Root node for the tree
*  If all Examples are positive, Return the single-node tree Root, with label = +
*  If all Examples are negative, Return the single-node tree Root, with label = -
*  If Attributes is empty, Return the single-node tree Root, with label = most common value of Target attribute in Examples
*  Otherwise Begin
    *  A <-- the attribute from Attributes that best* classifies Examples
    *  The decision attribute for Root <-- A
    *  For each possible value, vi, of A,
        *   Add a new tree branch below Root, corresponding to the test A = vi
        *   Let Examples: vi, be the subset of Examples that have value vi for A
        *   If Examples: vi, is empty
                * Then below this new branch add a leaf node with label = most common value of Target attribute in Examples
                * Else below this new branch add the subtree
                        ID3(Examples:vi, Target attribute, Attributes - {A})
*  End
*  Return Root

```





# Code

## Imports and Initialization

In [30]:
import pandas as pd
import numpy as np
import math

In [31]:
df = pd.read_csv("https://raw.githubusercontent.com/Ajith-Kumar-G/Clg-Labs/main/VI%20sem/ML_LAB/LAB%203/data.csv")
df = df.iloc[:,1:6]

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   outlook   14 non-null     object
 1   temp      14 non-null     object
 2   humidity  14 non-null     object
 3   wind      14 non-null     object
 4   play      14 non-null     object
dtypes: object(5)
memory usage: 688.0+ bytes


In [33]:
feature_names = df.columns[0:4].values
target = df.columns[-1]
print('Features in the dataset: ', feature_names)
print('Target in the dataset: ', target)

Features in the dataset:  ['outlook' 'temp' 'humidity' 'wind']
Target in the dataset:  play


### Visulaizing the dataset


In [74]:
import plotly.graph_objects as go
import plotly.express as px

In [75]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df.outlook, df.temp, df.humidity, df.wind, df.play],
               fill_color='lavender',
               align='left')
    )
])
fig.update_layout(title={'text': "Dataset:",
                         'y':0.9,
                         'font':dict(size=35),},
                font=dict(
                family="Trebuchet MS",
    ))
fig.show()

In [70]:
fig1 = px.scatter(df,x=['outlook', 'temp', 'humidity', 'wind'],title="Classification")
fig1.show()

In [73]:
fig2 = px.histogram(df, title= 'Count of features:')
fig2.show()

## Functions


In [ ]:
class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""

def entropy(data):
    pos = neg = 0
    if len(data[target].unique()) == 2:
        pos, neg = data[target].value_counts()
    if pos == 0 or neg == 0:
        return 0
    else:
        p = pos / (pos + neg)
        n = neg / (pos + neg)
        return -(p * math.log(p, 2) + n * math.log(n, 2))

def total_gain(data, attr):
    uniq = data[attr].unique()
    gain = entropy(data)
    for u in uniq:
        subdata = data[data[attr] == u]
        gain -= (float(len(subdata)) / float(len(data))) * entropy(subdata)
    return gain

def ID3(data, features):
    root = Node()
    max_gain = 0
    choo_feat = ""
    for feature in features:
        gain = total_gain(data, feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
    root.value = max_feat
    uniq = df[max_feat].unique()

    for u in uniq:
        subdata = data[data[max_feat] == u]
        newNode = Node()
        if entropy(subdata) == 0:
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = subdata[target].unique()
        else:
            newNode.value = u
            new_attrs = features.copy()
            new_attrs.remove(max_feat)
            child = ID3(subdata, new_attrs)
            newNode.children.append(child)
        root.children.append(newNode)
    return root

def printTree(root: Node, depth=0):
    print("\t"*depth,"|____", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" ---> ", root.pred)
    print()
    for child in root.children:
        printTree(child, depth + 1)

In [ ]:
root = ID3(df, feature_names.tolist())
printTree(root)

 |____outlook
	 |____Sunny
		 |____humidity
			 |____High --->  ['No']

			 |____Normal --->  ['Yes']

	 |____Overcast --->  ['Yes']

	 |____Rain
		 |____wind
			 |____Weak --->  ['Yes']

			 |____Strong --->  ['No']

